In [ ]:
import pandas as pd 
import seaborn as sns 
import numpy as np 
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

# **Exploratory Analysis - - - - - - - - - - - - Analisis Exploratorio**

**Loading and inspection   - - - - - - - - -  Carga e Inspección**

In [ ]:
dataset = pd.read_csv('../input/call-center-metrics-dataset/call_metrics_dataset.csv', delimiter = ';')
dataset.index = pd.to_datetime(dataset['date'])
del(dataset['date'])
dataset['avg_aht'] = dataset['avg_aht'].str.replace('.', '', regex = True)
dataset['avg_aht'] = dataset['avg_aht'].str.replace(',', '.', regex = True).astype(float)
dataset.head()

**Correlation - - - - - - - - - - - - - - Correlación**

In [ ]:
corr = dataset.astype('float64').corr()
ax = sns.heatmap(corr, annot = True)

**Splitting the dataset      - - - - - - - - - - - - - - -        Dividiendo el dataset**

In [ ]:
x = dataset.iloc[:, 0:6].values
y = dataset.iloc[:, -1]. values
x

**Dividing into training and testing and normalizing the variables - - - - - - - - - - - - - - Dividiendo en entrenamiento y test y normalizando las variables**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train, x_test = sc_x.fit_transform(x_train), sc_x.transform(x_test)

**Using variable reduction (PCA) - - - - - - - - - - - Usando reducción de variables (PCA)**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)
explained_variance = pca.explained_variance_ratio_
explained_variance

# **Selection Models- - - - - - - - - - - - - - - - - - - Selección de Modelos**

**XGBOOST CLASSIFIER**

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier(use_label_encoder = False, eval_metric = 'mlogloss').fit(x_train, y_train)
y_classifier = classifier.predict(x_test)
accuracy_score(y_test, y_classifier)

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators = 10).fit(x_train, y_train)
y_classifier2 = classifier2.predict(x_test)
accuracy_score(y_test, y_classifier2)

**SUPPORT VECTOR MACHINE**

In [ ]:
from sklearn.svm import SVC
classifier3 = SVC(gamma = 1).fit(x_train, y_train)
y_classifier3 = classifier3.predict(x_test)
accuracy_score(y_test, y_classifier3)

**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier4 = LogisticRegression(max_iter = 200).fit(x_train, y_train)
y_classifier4 = classifier4.predict(x_test)
accuracy_score(y_test, y_classifier4)

# **Learning curve**

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

**XGBOOST**

In [ ]:
title = "Curva de aprendizaje"
cv = ShuffleSplit(n_splits = 10, test_size = 0.3)
plot_learning_curve(classifier, title, x_train, y_train, ylim = (0.6, 1.0), cv = cv, n_jobs = 1)

**RANDOM FOREST**

In [ ]:
plot_learning_curve(classifier2, title, x_train, y_train, ylim = (0.6, 1.0), cv = cv, n_jobs = 1)

**SVM**

In [ ]:
plot_learning_curve(classifier3, title, x_train, y_train, ylim = (0.6, 1.0), cv = cv, n_jobs = 1)

**LOGISTIC REGRESSION**

In [ ]:
plot_learning_curve(classifier4, title, x_train, y_train, ylim = (0.6, 1.0), cv = cv, n_jobs = 1)

# **Conclusions - - - - - - - - - - - - - - - - Conclusiones**

**Ingles:**

*We can see that within our models a good performance is displayed, however this may be due to the few data we have and although all the models are giving us very similar predictions, we can observe that in the learning curves who is best adapting is XGBOOST, likewise we can highlight the performance that Logistic Regression is giving us although its accuracy is ~ 4% (approximate (~)) below XGBOOST, therefore the recommendation is that if more data were obtained one of the models to consider first will be XGBOOST*

**Español:**

*Podemos observar que dentro de nuestros modelos se visualiza un buen desempeño, sin embargo esto se puede deber a los pocos datos con los que contamos y aunque todos los modelos esten dandonos predicciones muy similares podemos observar que en las curvas de aprendizaje quien mejor se esta adaptando es XGBOOST, así mismo podemos resaltar el desempeño que nos esta dando Logistic Regression aunque su accuracy esta en un ~4% (aproximado(~)) por debajo de XGBOOST, por tanto la recomendación es que si se consiguieran más datos uno de los modelos a considerar primeramente sera XGBOOST*
